This notebook analyze the performance of the deberta model in QA model when Fold 0 is used for testing.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForQuestionAnswering
import os
from datasets import load_dataset
from transformers import pipeline
import sys
import evaluate
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
data_path = "../data/"
def load_data(fold):
    if fold == 0:
        dataset = load_dataset('json', data_files={
            'train': [data_path + 'subset-0.json', data_path + 'subset-1.json', data_path + 'subset-2.json',
                      data_path + 'subset-3.json'],
            'validation': data_path + 'subset-4.json'})
    elif fold == 1:
        dataset = load_dataset('json', data_files={
            'train': [data_path + 'subset-1.json', data_path + 'subset-2.json', data_path + 'subset-3.json',
                      data_path + 'subset-4.json'],
            'validation': data_path + 'subset-0.json'})
    elif fold == 2:
        dataset = load_dataset('json', data_files={
            'train': [data_path + 'subset-0.json', data_path + 'subset-2.json', data_path + 'subset-3.json',
                      data_path + 'subset-4.json'
                      ],
            'validation': data_path + 'subset-1.json'})
    elif fold == 3:
        dataset = load_dataset('json', data_files={
            'train': [data_path + 'subset-0.json', data_path + 'subset-1.json', data_path + 'subset-3.json',
                      data_path + 'subset-4.json'],
            'validation': data_path + 'subset-2.json'})
    elif fold == 4:
        dataset = load_dataset('json', data_files={
            'train': [data_path + 'subset-0.json', data_path + 'subset-1.json', data_path + 'subset-2.json',
                      data_path + 'subset-4.json'],
            'validation': data_path + 'subset-3.json'})

    return dataset


In [ ]:
#change standard output for this notebook
old_stdout = sys.stdout
sys.stdout= open("q3_deberta_fold4_WITHOUT_impossible_answers.txt","w")


In [ ]:

exp = 1
#load fold 1
data_s = load_data(exp)
val_data = data_s["validation"]#.select(range(100))

extracted_ds = set()
idss = list()
model_name = "deberta-base"

q_no = 3
#folder_name = f"analysis_dir/deberta-base-exp{exp}-q{q_no}-analysis"
question = "Which dataset or database is used?"
message= f"Here we check the performance of {model_name} with on 10 positive and 10 negative contentexts.#The model from experiment {exp} with question 3 is used."
#The model from experiment 1 with question q4 (i.q. q3) is used."
model_checkpoint =f"best_model/fold{exp}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


pipe = pipeline("question-answering",model=model,tokenizer = tokenizer,device=0)


metric = evaluate.load("squad_v2")

predicted_answers = []
theoretical_answers = []
result = []
for sample in tqdm(val_data):
    predicted_ans = pipe(question=sample["question"],context=sample["context"])#,handle_impossible_answer=True)

    result.append((predicted_ans["answer"],predicted_ans["start"],predicted_ans["end"],
                  sample["answers"]["text"],sample["answers"]["answer_start"],sample["id"]))
    if predicted_ans["start"] != predicted_ans["end"]:
        #(predicted,start,end,true,start)
        predicted_answers.append({"id": sample["id"], "prediction_text": predicted_ans["answer"].strip(),"no_answer_probability": 0.0})
    else:
        predicted_answers.append({"id": sample["id"], "prediction_text": "","no_answer_probability": 1.0})
        
    theoretical_answers.append({"id": sample["id"], "answers": sample["answers"]})

#print(predicted_answers))
print(extracted_ds)
print("\n*****************************************##################$$$$$$$$$$$$$$$$$$$$$$$$$$$\n")
#print(theoretical_answers)
print(len(idss))
print("__________________________________________________________________________________________")
print(metric.compute(predictions=predicted_answers, references=theoretical_answers))
print("________________________________________________________________________________________")
#print(match_list)
print("work is done2222222222222222222222222222222\n")



In [ ]:
# #This function reads the dataset id file into a dictionary
def read_dataset_ids_file(dataset_ids_file):
    dic = {}
    with open(dataset_ids_file,"r") as f:
        for line in f.readlines():
            line = line.split(":")
            ds_strings = line[1]
            ds_strings = ds_strings.split("$")
            dic[line[0].strip()] = set([x.strip() for x in ds_strings])

    return dic

In [ ]:
'''This function takes a dataset mention string and return its dataset id'''
def get_dataset_id(dataset_str,id_dict=None):
    if id_dict == None:
        id_dict = read_dataset_ids_file()
    for k,v in id_dict.items():
        if dataset_str in v:
            return k
    return None

In [ ]:
#old_stdout = sys.stdout
for exp in [1]: #every exp is on one fold
    #sys.stdout= open(f"qa_with_cls_as_probs/fold{exp}_with_impossible.txt","w")
    
    #load fold 1
    data_s = load_data(exp)
    val_data = data_s["validation"]#.select(range(100))

    extracted_ds = set()
    idss = list()
    model_name = "deberta-base"

    q_no = 3
    #folder_name = f"analysis_dir/deberta-base-exp{exp}-q{q_no}-analysis"
    question = "Which dataset or database is used?"
    message= f"Here we check the performance of {model_name} with positive and  negative contentexts.The model from experiment {exp} with question 3 is used with classification as probability for the no_answer"
    #The model from experiment 1 with question q4 (i.q. q3) is used."
    model_checkpoint =f"best_model/fold{exp}"
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


    pipe = pipeline("question-answering",model=model,tokenizer = tokenizer,device=0)


    metric = evaluate.load("squad_v2")

    predicted_answers = []
    theoretical_answers = []
    result = []
    for sample in tqdm(val_data):
        predicted_ans = pipe(question=sample["question"],context=sample["context"],handle_impossible_answer=True)

        result.append((predicted_ans["answer"],predicted_ans["start"],predicted_ans["end"],
                      sample["answers"]["text"],sample["answers"]["answer_start"],sample["id"]))
        if predicted_ans["start"] != predicted_ans["end"]:
            #(predicted,start,end,true,start)
            predicted_answers.append({"id": sample["id"], "prediction_text": predicted_ans["answer"].strip(),"no_answer_probability": 0.0})
        else:
            predicted_answers.append({"id": sample["id"], "prediction_text": "","no_answer_probability": 1.0})

        theoretical_answers.append({"id": sample["id"], "answers": sample["answers"]})

    #print(predicted_answers))
    print(extracted_ds)
    print("\n*****************************************##################$$$$$$$$$$$$$$$$$$$$$$$$$$$\n")
    #print(theoretical_answers)
    print(len(idss))
    print("__________________________________________________________________________________________")
    print(metric.compute(predictions=predicted_answers, references=theoretical_answers))
    print("________________________________________________________________________________________")
    #print(match_list)  

In [ ]:
#p positive n negative
#first is true and second is predicted
tp = []
fn = []
fp= []
tn= []
for x in result:                                                
    if len(x[3]) >0 and x[0] != '':#tp                               
        tp.append(x)
    elif len(x[3]) >0 and x[0] == '': #fn                          
        fn.append(x)                                                
    elif len(x[3]) ==0 and x[0] != '':#fp                           
        fp.append(x)                                                
    elif len(x[3]) == 0 and x[0] =='':#tn                       
        tn.append(x) 
        
print(f"TP: {len(tp)}, FN: {len(fn)}, FP: {len(fp)}, TN: {len(tn)}\n")
print("**********************************************************************")

print("True Positive\n")
tp_ex = {}
for x in tp:
    ds_str = x[3][0]
    if ds_str not in tp_ex.keys():
        tp_ex[ds_str] =1
    else:
        tp_ex[ds_str] +=1
for k,v in tp_ex.items():
    print("{0},{1}\n".format(k,v))
print("**********************************************************************") 
print("False Negative")
fn_ex = {}
for x in fn:
    ds_str = x[3][0]
    if ds_str not in fn_ex.keys():
        fn_ex[ds_str] =1
    else:
        fn_ex[ds_str] +=1
for k,v in fn_ex.items():
    print("{0},{1}\n".format(k,v))

print("**********************************************************************")
print("False Positive\n")
fp_ex = {}
for x in fp:
    print(x)
    ds_str = x[0]
    if ds_str not in fp_ex.keys():
        fp_ex[ds_str] =1
    else:
        fp_ex[ds_str] +=1
for k,v in fp_ex.items():
    print("{0},{1}\n".format(k,v))

In [ ]:
#change standard output again
sys.stdout = old_stdout

In [ ]:
dataset_file = "/media/nvme1n1/yyounes/paper_1/data/datasets_final.txt"

ds_dict = read_dataset_ids_file(dataset_file)
tp_dict = {}
print(len(ds_dict))

trash = []
for x in tp:
    ds_id = get_dataset_id(dataset_str=x[0].strip(),id_dict=ds_dict)
    if ds_id != None and int(ds_id) in valid_ds_ids:
        if ds_id not in tp_dict:
            tp_dict[ds_id]=1
        else: #if ds_id != None:
            tp_dict[ds_id]+=1
    else:
        trash.append(ds_id)
print("Trash ids")
print(trash)
print("************************")
tp_dict['5'] = 0 #add this because dataset 5 does not appear in the results although it is in the gold standard
sorted_dict = {k: v for k, v in sorted(tp_dict.items(), key=lambda item: int(item[0]),reverse=False)}
#sorted_dict = dict(sorted(tp_dict.items(),reverse=True))
id_list = list(sorted_dict.keys())
count_list = list(sorted_dict.values())
print(id_list)
print(count_list)


In [ ]:
for x in tp:
    pred_start = x[1]
    gold_start = list(x[4])
    print("index difference")
    #print(f"{x[0]}:{gold_start[0]-pred_start}")


In [ ]:

valid_ds_ids=[6, 18, 2, 25, 12, 5, 31, 30]

fn_dict = {}
for fff in valid_ds_ids:
    fn_dict[str(fff)]=0
    
for x in fn:
    ds_str = list(x[3])[0] #get dataset gold standard string
    ds_id = get_dataset_id(ds_str,ds_dict)
    if ds_id == '18':
        print(ds_str)
        print("****************")
    if ds_id in fn_dict:
        fn_dict[ds_id]+=1
    #else:
        #fn_dict[ds_id]+=1
        
        
#fn_dict['30']=0
#fn_dict['31']=0
print(fn_dict[str(18)])
sorted_dict_fn = {k: v for k, v in sorted(fn_dict.items(), key=lambda item: int(item[0]),reverse=False)}
#sorted_dict = dict(sorted(tp_dict.items(),reverse=True))
id_list_fn= list(sorted_dict_fn.keys())
count_list_fn = list(sorted_dict_fn.values())
print(id_list_fn)
print(count_list_fn)
#count_list_fn = [-1*x for x in count_list_fn]
#id_list.append('99-new')
#count_list.append(29)
print(count_list)
print(id_list)
original_support = [138,2,13184,107,138,128,1,1]
fig, ax = plt.subplots()    
width = 0.4 # the width of the bars 
#ind = np.arange(len(id_list))  # the x locations for the groups
#ax.bar(id_list_fn, count_list_fn, width, color="orange")
ax.bar(id_list,count_list,width,color="blue")
#ax.set_yticks(ind+width/2)
#ax.set_yticklabels(x, minor=False)
for i, o_sup,c_tp in zip(id_list,original_support,count_list):
    if c_tp == 0:
        print(i)
    lb = f'{c_tp}/{o_sup}'
    #ax.text(i, c_fn-500, str(c_fn), color='black', fontweight='bold')#,rotation=90)
    ax.text(i,c_tp+15,lb,color='black',fontweight='bold')
#for i, v in enumerate(count_list):
#    ax.text(v + 3, i, str(v), color='black', fontweight='bold')
#plt.title('Datasets Found in Fold 0')
plt.ylabel('Number of datasets')
plt.xlabel('Dataset ID')      
#plt.show()
fig.set_size_inches(6, 4)
fig.savefig("fold0_ids_found.pdf",dpi=300)
#fig.savefig(os.path.join('fold0_ids_found.jpg'), dpi=300, format='pddf', bbox_inches='tight') 